In [1]:
import requests

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 1990) 

In [3]:
from bs4 import BeautifulSoup

In [4]:
#ASSIGN variable url as our Wikipedia link
url = "https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A3%D0%BC%D0%B5%D1%80%D1%88%D0%B8%D0%B5_%D0%BE%D1%82_COVID-19_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"

In [5]:
#READ the url
response = requests.get(url)

In [6]:
#GET the content with BS
soup = BeautifulSoup(response.text, "html.parser")

In [7]:
#FIND persons as a list on a page
persons = soup.select('a', class_="mw-content-ltr", id='bodyContent')[34:234]

In [8]:
#GET links for each person
links =[]
for person in persons:
    links.append(person.get('href'))

In [9]:
#ADD beginning of the url to get the urls
urls = []
for link in links:
    urls.append("https://ru.wikipedia.org"+link)

In [10]:
#GET names of each person
names = []
for person in persons:
    names.append(person.get('title'))

In [11]:
#CREATE a dataframe with urls as columns
df = pd.DataFrame(urls, columns = ['urls'])

In [12]:
#ADD column with names 
df['names']= names
df.shape

(200, 2)

In [13]:
#GET the content from the second Wikipedia page
url1 = "https://ru.wikipedia.org/w/index.php?title=%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%A3%D0%BC%D0%B5%D1%80%D1%88%D0%B8%D0%B5_%D0%BE%D1%82_COVID-19_%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8&pagefrom=%D0%A8%D1%83%D0%B2%D0%B0%D0%BB%D0%BE%D0%B2%2C+%D0%92%D0%B8%D0%BA%D1%82%D0%BE%D1%80+%D0%93%D1%80%D0%B8%D0%B3%D0%BE%D1%80%D1%8C%D0%B5%D0%B2%D0%B8%D1%87#mw-pages"

In [14]:
#READ the url
response = requests.get(url1)

In [15]:
#GET the content with BS
soup = BeautifulSoup(response.text, "html.parser")

In [16]:
#FIND persons as a list on a page
persons = soup.select('a', class_="mw-content-ltr", id='bodyContent')[34:42]

In [17]:
#GET links for each person
links =[]
for person in persons:
    links.append(person.get('href'))

In [18]:
#ADD beginning of the url to get the urls
urls = []
for link in links:
    urls.append("https://ru.wikipedia.org"+link)

In [19]:
#GET names of each person
names = []
for person in persons:
    names.append(person.get('title'))

In [20]:
#CREATE a dataframe with urls as columns
df1 = pd.DataFrame(urls, columns = ['urls'])

In [21]:
#ADD column with names 
df1['names']= names
df1.shape

(8, 2)

In [22]:
frames = [df, df1]
df = pd.concat(frames)
df.shape

(208, 2)

In [23]:
df['contents'] = df.urls.apply(lambda url: requests.get(url).text)

In [24]:
def get_bio(content):
    soup = BeautifulSoup(content)
    bio = soup.select_one('p', id="Биография")
    return bio.text.strip()

In [25]:
df['bio']=df.contents.apply(get_bio)

In [111]:
def get_birth(content):
    soup = BeautifulSoup(content)
    try:
        tables = soup.select("table")
        trs = tables[0].find_all('td', class_='plainlist')
        td = trs[0].text.strip()
        return (td)
    except:
        pass

In [113]:
df['birth']=df.contents.apply(get_birth)

In [119]:
def get_death(content):
    try:
        soup = BeautifulSoup(content)
        tables = soup.select("table")
        trs = tables[0].find_all('td', class_='plainlist')
        td = trs[2].text.strip()
        return (td)
    except:
        pass

In [120]:
df['death']=df.contents.apply(get_death)

In [122]:
def get_place(content):
    try:
        soup = BeautifulSoup(content)
        tables = soup.select("table")
        trs = tables[0].find_all('td', class_='plainlist')
        td = trs[3].text.strip()
        return (td)
    except:
        pass

In [124]:
df['place'] = df.contents.apply(get_place)

In [126]:
df1=df
del df1['contents']

In [127]:
df1.to_csv('people.csv', index = False)